# T1207 Rogue Domain Controller

-----------------------------------------------------------------------

## Technique Description

Adversaries may register a rogue Domain Controller to enable manipulation of Active Directory data. DCShadow may be used to create a rogue Domain Controller (DC). DCShadow is a method of manipulating Active Directory (AD) data, including objects and schemas, by registering (or reusing an inactive registration) and simulating the behavior of a DC. (Citation: DCShadow Blog) Once registered, a rogue DC may be able to inject and replicate changes into AD infrastructure for any domain object, including credentials and keys.

Registering a rogue DC involves creating a new server and nTDSDSA objects in the Configuration partition of the AD schema, which requires Administrator privileges (either Domain or local to the DC) or the KRBTGT hash. (Citation: Adsecurity Mimikatz Guide)

This technique may bypass system logging and security monitors such as security information and event management (SIEM) products (since actions taken on a rogue DC may not be reported to these sensors). (Citation: DCShadow Blog) The technique may also be used to alter and delete replication and other associated metadata to obstruct forensic analysis. Adversaries may also utilize this technique to perform [SID-History Injection](https://attack.mitre.org/techniques/T1134/005) and/or manipulate AD objects (such as accounts, access control lists, schemas) to establish backdoors for Persistence. (Citation: DCShadow Blog)

## Technique Detection

Monitor and analyze network traffic associated with data replication (such as calls to DrsAddEntry, DrsReplicaAdd, and especially GetNCChanges) between DCs as well as to/from non DC hosts. (Citation: GitHub DCSYNCMonitor) (Citation: DCShadow Blog) DC replication will naturally take place every 15 minutes but can be triggered by an adversary or by legitimate urgent changes (ex: passwords). Also consider monitoring and alerting on the replication of AD objects (Audit Detailed Directory Service Replication Events 4928 and 4929). (Citation: DCShadow Blog)

Leverage AD directory synchronization (DirSync) to monitor changes to directory state using AD replication cookies. (Citation: Microsoft DirSync) (Citation: ADDSecurity DCShadow Feb 2018)

Baseline and periodically analyze the Configuration partition of the AD schema and alert on creation of nTDSDSA objects. (Citation: DCShadow Blog)

Investigate usage of Kerberos Service Principal Names (SPNs), especially those associated with services (beginning with “GC/”) by computers not present in the DC organizational unit (OU). The SPN associated with the Directory Replication Service (DRS) Remote Protocol interface (GUID E3514235–4B06–11D1-AB04–00C04FC2DCD2) can be set without logging. (Citation: ADDSecurity DCShadow Feb 2018) A rogue DC must authenticate as a service using these two SPNs for the replication process to successfully complete.

-----------------------------------------------------------------------

### Tactics:

  *   Defense-Evasion

### Platforms:

  * Windows

### Adversary Required Permissions:

  * Administrator

### Defenses Bypassed:

  * Log analysis

### Data Sources:

  * **Network Traffic:** Network Traffic Content

  * **Active Directory:** Active Directory Object Creation

  * **Active Directory:** Active Directory Object Modification

  * **User Account:** User Account Authentication

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1207)

  * [Dcshadow Blog](https://www.dcshadow.com/), Delpy, B. & LE TOUX, V. (n.d.). DCShadow. Retrieved March 20, 2018.

  * [Adsecurity Mimikatz Guide](https://adsecurity.org/?page_id=1821), Metcalf, S. (2015, November 13). Unofficial Guide to Mimikatz & Command Reference. Retrieved December 23, 2015.

  * [Github Dcsyncmonitor](https://github.com/shellster/DCSYNCMonitor), Spencer S. (2018, February 22). DCSYNCMonitor. Retrieved March 30, 2018.

  * [Microsoft Dirsync](https://msdn.microsoft.com/en-us/library/ms677626.aspx), Microsoft. (n.d.). Polling for Changes Using the DirSync Control. Retrieved March 30, 2018.

  * [Addsecurity Dcshadow Feb 2018](https://adds-security.blogspot.fr/2018/02/detecter-dcshadow-impossible.html), Lucand,G. (2018, February 18). Detect DCShadow, impossible?. Retrieved March 30, 2018.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversaries will register a rogue Domain Controller to enable manipulation of Active Directory data.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

#### APT28, Turla	
- Mimikatz’s LSADUMP::DCShadow module can be used to make AD updates by temporarily setting a computer to be a DC.


## Detection Blindspots

- Information Here

## Analytical References

  * [DC Shadow (dcshadow.com)](https://www.dcshadow.com/)
  * [adsecurity.org](https://adsecurity.org/?page_id=1821)
  * [Mimikatz (github)](https://github.com/gentilkiwi/mimikatz)
  * [DCSyncMonitor (github)](https://github.com/shellster/DCSYNCMonitor)
  * [Polling for Changes Using the DirSync Control (microsoft)](https://docs.microsoft.com/en-us/windows/win32/ad/polling-for-changes-using-the-dirsync-control?redirectedfrom=MSDN)
  * [Windows Server 2008 R2 and 2008 (microsoft)](https://docs.microsoft.com/en-us/previous-versions/windows/it-pro/windows-server-2008-R2-and-2008/dd941628(v=ws.10))

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Using logs, DCShadow can be detected when objects in the Configuration partition are added or when the computer object is changed. 
- Leverage AD directory synchronization (DirSync) to monitor changes to directory state using AD replication cookies. 
- Baseline and periodically analyze the Configuration partition of the AD schema and alert on creation of nTDSDSA objects. 
- Investigate usage of Kerberos Service Principal Names (SPNs), especially those associated with services (beginning with "GC/") by computers not present in the DC organizational unit (OU). The SPN associated with the Directory Replication Service (DRS) Remote Protocol interface (GUID E3514235–4B06–11D1-AB04–00C04FC2DCD2) can be set without logging. A rogue DC must authenticate as a service using these two SPNs for the replication process to successfully complete.

#### Analytic 1

  * **Information:** Monitor LSADump and DCShadow

  * **Source:** Windows Audits, Sysmon

  * **Tool:** Kibana

  * **Notes:** 

  * **Query:** ```process.command_line : *LSADUMP* OR *DCShadow*```

#### Analytic 2 

  * **Information:** Monitor Audit Detailed Directory Service Replication

  * **Source:** Windows Audits, Sysmon

  * **Tool:** Kibana

  * **Notes:** 
      - 4928 An Active Directory replica source naming context was established. 
      - 4929 An Active Directory replica source naming context was removed.
  * **Query:** ```event.code : 4928 OR 4929```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Monitor and analyze network traffic associated with data replication (such as calls to DrsAddEntry, DrsReplicaAdd, and especially GetNCChanges) between DCs as well as to/from non DC hosts. DC replication will naturally take place every 15 minutes but can be triggered by an attacker or by legitimate urgent changes (ex: passwords). Also consider monitoring and alerting on the replication of AD objects (Audit Detailed Directory Service Replication Events 4928 and 4929).
- DCShadow is easy to detect at network level. API like DrsAddEntry or DrsReplicaAdd are called only from a DC so a call from another computer should be considered as suspicious.

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```
